# AI Config API - Cookbook

This cookbook contains all code from `aiconfig-api/SKILL.md` for API access patterns and authentication.

## Prerequisites
- `LAUNCHDARKLY_API_TOKEN`: API token with appropriate permissions

In [1]:
%pip install requests python-dotenv


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load environment variables from .env
import os
from dotenv import load_dotenv

def find_repo_root():
    """Find repo root by looking for .git directory"""
    path = os.getcwd()
    while path != "/":
        if os.path.exists(os.path.join(path, ".git")):
            return path
        path = os.path.dirname(path)
    return os.getcwd()

REPO_ROOT = find_repo_root()
ENV_FILE = os.path.join(REPO_ROOT, ".env")
load_dotenv(ENV_FILE)
print(f"[OK] Loaded environment from: {ENV_FILE}")

[OK] Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


---
## API Authentication
From: `SKILL.md` lines 47-55

In [3]:
import requests
import os

headers = {
    "Authorization": os.environ.get("LAUNCHDARKLY_API_TOKEN"),
    "Content-Type": "application/json",
    "LD-API-Version": "beta"  # Required for AI Config endpoints
}
print("[OK] Headers configured")

[OK] Headers configured


---
## List AI Configs
From: `SKILL.md` lines 84-104

In [4]:
def list_ai_configs(project_key, api_token):
    """List all AI Configs in a project"""
    url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs"

    headers = {
        "Authorization": api_token,
        "LD-API-Version": "beta"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        configs = response.json()
        for config in configs.get("items", []):
            print(f"- {config['key']}: {config['name']}")
        return configs
    else:
        print(f"Error: {response.status_code}")
        return None

---
## Verify: list_ai_configs()

In [5]:
# Test list_ai_configs()
configs = list_ai_configs("support-ai", os.environ.get("LAUNCHDARKLY_API_TOKEN"))
print(f"\nFound {len(configs.get('items', [])) if configs else 0} AI Configs")

- content-assistant: Content Generation Assistant
- ld-ai-judge-accuracy-1770164301356: AI Judge - Accuracy
- ld-ai-judge-relevance-1770164301550: AI Judge - Relevance
- ld-ai-judge-toxicity-1770164301695: AI Judge - Toxicity
- support-agent: Customer Support Agent

Found 5 AI Configs


---
## Error Handling
From: `SKILL.md` lines 123-153

In [6]:
def api_request(method, url, headers, json_data=None):
    """Make API request with error handling"""
    try:
        if method == "GET":
            response = requests.get(url, headers=headers)
        elif method == "POST":
            response = requests.post(url, headers=headers, json=json_data)
        elif method == "PATCH":
            response = requests.patch(url, headers=headers, json=json_data)
        elif method == "DELETE":
            response = requests.delete(url, headers=headers)

        if response.status_code in [200, 201, 204]:
            return {"success": True, "data": response.json() if response.text else None}
        elif response.status_code == 400:
            return {"success": False, "error": "Bad request - check your data"}
        elif response.status_code == 401:
            return {"success": False, "error": "Invalid or missing API token"}
        elif response.status_code == 403:
            return {"success": False, "error": "Insufficient permissions"}
        elif response.status_code == 404:
            return {"success": False, "error": "Resource not found"}
        elif response.status_code == 429:
            return {"success": False, "error": "Rate limited - wait and retry"}
        else:
            return {"success": False, "error": f"HTTP {response.status_code}"}

    except requests.exceptions.RequestException as e:
        return {"success": False, "error": str(e)}

---
## Verify: api_request()

In [7]:
# Test api_request() with a GET request
api_token = os.environ.get("LAUNCHDARKLY_API_TOKEN")
test_url = "https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs"
test_headers = {"Authorization": api_token, "LD-API-Version": "beta"}

result = api_request("GET", test_url, test_headers)
print(f"Success: {result['success']}")
if result['success']:
    print(f"Found {len(result['data'].get('items', []))} configs")
else:
    print(f"Error: {result.get('error')}")

Success: True
Found 5 configs


---
## Pagination
From: `SKILL.md` lines 159-185

In [8]:
def get_all_configs(project_key, api_token):
    """Get all AI Configs with pagination"""
    all_configs = []
    limit = 20
    offset = 0

    while True:
        url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs"
        url += f"?limit={limit}&offset={offset}"

        result = api_request("GET", url, {"Authorization": api_token, "LD-API-Version": "beta"})

        if not result["success"]:
            break

        items = result["data"].get("items", [])
        all_configs.extend(items)

        # Check if more pages exist
        if len(items) < limit:
            break

        offset += limit

    return all_configs

---
## Verify: get_all_configs()

In [9]:
# Test get_all_configs()
all_configs = get_all_configs("support-ai", os.environ.get("LAUNCHDARKLY_API_TOKEN"))
print(f"Found {len(all_configs)} total AI Configs (with pagination)")
for config in all_configs:
    print(f"  - {config['key']}: {config['name']}")

Found 5 total AI Configs (with pagination)
  - content-assistant: Content Generation Assistant
  - ld-ai-judge-accuracy-1770164301356: AI Judge - Accuracy
  - ld-ai-judge-relevance-1770164301550: AI Judge - Relevance
  - ld-ai-judge-toxicity-1770164301695: AI Judge - Toxicity
  - support-agent: Customer Support Agent


---
## Rate Limiting with Retry
From: `SKILL.md` lines 215-234

In [10]:
import time

def retry_with_backoff(func, max_retries=3):
    """Retry function with exponential backoff"""
    for attempt in range(max_retries):
        result = func()

        if result.get("success"):
            return result

        if "rate limited" in result.get("error", "").lower():
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"Rate limited, waiting {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            return result

    return {"success": False, "error": "Max retries exceeded"}

---
## Verify: retry_with_backoff()

In [11]:
# Test retry_with_backoff() with a simple API call
api_token = os.environ.get("LAUNCHDARKLY_API_TOKEN")

def test_api_call():
    url = "https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs"
    return api_request("GET", url, {"Authorization": api_token, "LD-API-Version": "beta"})

result = retry_with_backoff(test_api_call)
print(f"Success: {result['success']}")
if result['success']:
    print(f"Found {len(result['data'].get('items', []))} configs")

Success: True
Found 5 configs


---
## Quick Reference: Essential Headers
From: `SKILL.md` lines 240-246

In [12]:
headers = {
    "Authorization": "YOUR_API_TOKEN",     # Required
    "Content-Type": "application/json",    # For POST/PATCH
    "LD-API-Version": "beta"               # For AI Config endpoints
}

---
## Quick Reference: Base URLs
From: `SKILL.md` lines 250-255

In [13]:
# Choose based on your environment
BASE_URL = "https://app.launchdarkly.com/api/v2"  # Production
# BASE_URL = "https://app.launchdarkly.us/api/v2"  # Federal
# BASE_URL = "https://app.eu.launchdarkly.com/api/v2"  # EU

---
## Quick Reference: Common Endpoints
From: `SKILL.md` lines 259-287

In [14]:
project = "support-ai"
env = "production"
key = "my-config"
var_key = "my-variation"
metric_key = "my-metric"
segment_key = "my-segment"

# AI Configs
print(f"{BASE_URL}/projects/{project}/ai-configs")  # List/Create
print(f"{BASE_URL}/projects/{project}/ai-configs/{key}")  # Get/Update/Delete

# Variations
print(f"{BASE_URL}/projects/{project}/ai-configs/{key}/variations")  # List/Create
print(f"{BASE_URL}/projects/{project}/ai-configs/{key}/variations/{var_key}")  # Get/Update/Delete

# Tools
print(f"{BASE_URL}/projects/{project}/ai-tools")  # List/Create
print(f"{BASE_URL}/projects/{project}/ai-tools/{key}")  # Get/Update/Delete

# AI Config Metrics (read AI usage metrics)
print(f"{BASE_URL}/projects/{project}/ai-configs/{key}/metrics?from=0&to=0&env={env}")  # Get

# Custom Metrics (manage metric definitions)
print(f"{BASE_URL}/metrics/{project}")  # List/Create
print(f"{BASE_URL}/metrics/{project}/{metric_key}")  # Get/Update/Delete

# Segments (for targeting)
print(f"{BASE_URL}/segments/{project}/{env}")  # List/Create
print(f"{BASE_URL}/segments/{project}/{env}/{segment_key}")  # Get/Update/Delete

# Projects
print(f"{BASE_URL}/projects")  # List/Create
print(f"{BASE_URL}/projects/{project}")  # Get/Update/Delete
print(f"{BASE_URL}/projects/{project}/environments")  # List environments

https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs/my-config
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs/my-config/variations
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs/my-config/variations/my-variation
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-tools
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-tools/my-config
https://app.launchdarkly.com/api/v2/projects/support-ai/ai-configs/my-config/metrics?from=0&to=0&env=production
https://app.launchdarkly.com/api/v2/metrics/support-ai
https://app.launchdarkly.com/api/v2/metrics/support-ai/my-metric
https://app.launchdarkly.com/api/v2/segments/support-ai/production
https://app.launchdarkly.com/api/v2/segments/support-ai/production/my-segment
https://app.launchdarkly.com/api/v2/projects
https://app.launchdarkly.com/api/v2/projects/support-ai
https://app.launchdarkly.com/api/v2/projects/supp

---
## Summary

In [15]:
print("=" * 50)
print("All aiconfig-api tests completed!")
print("=" * 50)
print("\n[OK] list_ai_configs() - Lists configs in project")
print("[OK] api_request() - Error handling wrapper")
print("[OK] get_all_configs() - Pagination support")
print("[OK] retry_with_backoff() - Rate limit handling")
print("[OK] Headers, Base URLs, and Endpoints documented")
print("\n[OK] All aiconfig-api tests pass!")

All aiconfig-api tests completed!

[OK] list_ai_configs() - Lists configs in project
[OK] api_request() - Error handling wrapper
[OK] get_all_configs() - Pagination support
[OK] retry_with_backoff() - Rate limit handling
[OK] Headers, Base URLs, and Endpoints documented

[OK] All aiconfig-api tests pass!
